First of all install library with `!pip install` command:

In [84]:
!pip install ipython-sql

In [85]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [86]:
!env | grep POST

POSTGRESQL_USER=reader
POSTGRESQL_PASSWORD=Miba2021


In [87]:
import os
USER = os.environ['POSTGRESQL_USER']
PASSWORD = os.environ['POSTGRESQL_PASSWORD']
POSTGRESQL_HOST = '10.129.0.25'
DBASE_NAME = 'demo'

In [88]:
CONNECT_DATA = 'postgresql://{}:{}@{}/{}'.format(
    USER,
    PASSWORD,
    POSTGRESQL_HOST,
    DBASE_NAME
)

In [10]:
print(result.keys)

RMKeyView(['aircraft_code', 'seat_no', 'fare_conditions'])


1. Which fare conditions are available for flights from AER to SVO (i.e. from Adler/Sochi to Sheremetevo/Moscow)

In [118]:
%%sql $CONNECT_DATA
SELECT distinct fare_conditions
FROM ticket_flights 
WHERE flight_id IN (
    SELECT distinct flight_id 
    FROM flights 
    WHERE departure_airport = 'AER' and arrival_airport='SVO')

3 rows affected.


fare_conditions
Business
Comfort
Economy


2. A list of aircraft models which fly to the Yakutsk AND Abakan cities

In [110]:
%%sql $CONNECT_DATA

SELECT DISTINCT model
FROM aircrafts
WHERE aircraft_code IN (
    SELECT aircraft_code 
    FROM flights 
    WHERE arrival_airport IN (
        SELECT airport_code
        FROM airports
        WHERE city = 'Yakutsk'
    )
)
AND aircraft_code IN (
    SELECT aircraft_code 
    FROM flights 
    WHERE arrival_airport IN (
        SELECT airport_code
        FROM airports
        WHERE city = 'Abakan'
    )
)

1 rows affected.


model
Airbus A319-100


3. A list of aircraft models which fly to the Yakutsk OR Abakan cities

In [111]:
%%sql $CONNECT_DATA

SELECT DISTINCT model
FROM aircrafts
WHERE aircraft_code IN (
    SELECT DISTINCT aircraft_code 
    FROM flights 
    WHERE arrival_airport IN (
        SELECT airport_code
        FROM airports
        WHERE city IN ('Yakutsk', 'Abakan')
    )
)

5 rows affected.


model
Airbus A319-100
Boeing 737-300
Bombardier CRJ-200
Cessna 208 Caravan
Sukhoi Superjet-100


4. A list of aircrafts which DON’T fly to Yakutsk city

In [113]:
%%sql $CONNECT_DATA
SELECT DISTINCT model
FROM aircrafts
WHERE aircraft_code not in (
    SELECT DISTINCT aircraft_code 
    FROM flights 
    WHERE arrival_airport in (
        SELECT airport_code
        FROM airports
        WHERE city = 'Yakutsk'
    )
)

6 rows affected.


model
Airbus A320-200
Airbus A321-200
Boeing 737-300
Boeing 767-300
Boeing 777-300
Cessna 208 Caravan


5. Find out airport codes related to Mrs. 'ELLA DMITRIEVA'  - from where and to where she flew.

In [116]:
%%sql $CONNECT_DATA
SELECT arrival_airport, departure_airport
FROM flights
WHERE flight_id in (
    SELECT flight_id
    FROM ticket_flights 
    WHERE ticket_no in (
        SELECT ticket_no
        FROM tickets
    WHERE tickets.passenger_name = 'ELLA DMITRIEVA')
)

7 rows affected.


arrival_airport,departure_airport
SVX,SVO
SVX,MQF
MQF,SVX
SVO,SVX
HMA,DME
UUS,DME
DME,HMA
